In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
from collections import Counter

### Read our dataframe

In [2]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/409319/test1_completed.csv')

### Let's check the structure of our df

In [3]:
df.head()

,id,Товар,Количество
0,17119,Лимон,1.1
1,17119,Лимон оранжевый,0.7
2,17119,Лук-порей,10.0
3,17119,Лук репчатый,2.5
4,17119,Малина свежая,1.0


In [4]:
initial_df_shape = df.shape

#### Let's rename columns 

In [5]:
df = df.rename(columns={"id": "order_id", "Товар": "goods", "Количество": "quantity"})

#### I'm just wondering if we have some orders with only one purchase

In [6]:
duplicated_df = df[df['order_id'].duplicated(keep=False)]

In [7]:
duplicated_df_shape = duplicated_df.shape

In [8]:
initial_df_shape[0] - duplicated_df_shape[0] # Looks like we have some orders with just one purchsed item 

139

#### But I'd like to check this in one more way just to be sure

In [9]:
non_duplicated = df.groupby('order_id').count().query('goods == 1')

In [10]:
non_duplicated.shape # Yes, thats true. We have some bunch of such orders

(139, 2)

#### Now we need to place all purchased items for eache order in one line 

In [11]:
duplicated_df['all_goods'] =  duplicated_df.groupby('order_id')['goods'].transform(lambda x: ','.join(x))

/Users/aleksandrzlenko/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
duplicated_df.head()

,order_id,goods,quantity,all_goods
0,17119,Лимон,1.1,"Лимон,Лимон оранжевый,Лук-порей,Лук репчатый,М..."
1,17119,Лимон оранжевый,0.7,"Лимон,Лимон оранжевый,Лук-порей,Лук репчатый,М..."
2,17119,Лук-порей,10.0,"Лимон,Лимон оранжевый,Лук-порей,Лук репчатый,М..."
3,17119,Лук репчатый,2.5,"Лимон,Лимон оранжевый,Лук-порей,Лук репчатый,М..."
4,17119,Малина свежая,1.0,"Лимон,Лимон оранжевый,Лук-порей,Лук репчатый,М..."


#### Now we need to drop duplicated values 

In [13]:
final_df = duplicated_df[['order_id', 'all_goods']].drop_duplicates()

In [14]:
final_df.head()

,order_id,all_goods
0,17119,"Лимон,Лимон оранжевый,Лук-порей,Лук репчатый,М..."
7,17530,"Лимон оранжевый,Изюм Султана,Капуста цветная,Б..."
19,17618,"Абрикосы молдавские,Кукуруза мини,Грибы лисичк..."
41,17724,"Абрикосы молдавские,Арбуз,Салат Айсберг,Галанг..."
55,17814,"Арбуз,Кабачки,Кинза,Малина свежая,Морковь мыта..."


#### Okay, let's make the final step

In [22]:
count = Counter()

for row in final_df['all_goods']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list, 2)))

for key, value in count.most_common(10):
    print(key, value)


('Огурцы Луховицкие', 'Укроп') 403
('Петрушка', 'Укроп') 390
('Арбуз', 'Огурцы Луховицкие') 314
('Кабачки', 'Огурцы Луховицкие') 300
('Кинза', 'Укроп') 290
('Лук зеленый', 'Укроп') 287
('Кабачки', 'Укроп') 267
('Арбуз', 'Укроп') 266
('Баклажаны грунтовые', 'Кабачки') 264
('Огурцы Луховицкие', 'Петрушка') 260


#### Some conclusions

In [ ]:
# It seems we are selling vegatables and fruits
# Looks like the most common goods sold together are vegetables 
# Realy supprised not to see tomatos in this list 